In [1]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

In [ ]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [ ]:
# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [ ]:
# Consulta SQL para obtener datos de la tabla de solicitudes de servicio
query = """
SELECT 
    solicitud_id,
    fecha_solicitud,
    cliente_id,
    sede_id,
    mensajero_id,
    tiempo_entrega
FROM solicitudes_servicio
"""

In [ ]:
# Leer datos de la fuente
df_solicitudes = pd.read_sql(query, fuente_conn)

In [ ]:
# Crear tabla de hechos Entrega_Servicio_Diaria
df_entrega_diaria = df_solicitudes.groupby(['solicitud_id', 'fecha_solicitud', 'cliente_id', 'sede_id', 'mensajero_id'])['tiempo_entrega'].sum().reset_index()


In [ ]:
# Cargar en la bodega de datos
df_entrega_diaria.to_sql('fact_entrega_servicio_diaria', bodega_conn, if_exists='replace', index=False)


In [ ]:
# Verificaciones adicionales
print("\nNúmero total de registros en la tabla de hechos Entrega_Servicio_Diaria:", len(df_entrega_diaria))
print("\nMuestra de los datos:")
print(df_entrega_diaria.head())